# Project 02: College Scorecard Data

### Due Date (Checkpoint): Wed 02/20/19 at Noon

### Due Date (Final): Wed 02/27/19 at Noon

---

This project consists of **two deadlines**:

* There is a **CHECKPOINT DEADLINE on Wednesday Feb 20 at Noon**. The problems due for the checkpoint are **problems 1,2,4,5**. While this deadline is the day before the midterm, these questions were written to be good practice for the midterm! They will be graded with "easier" tests and will be graded more thoroughly upon final submission. Submit the checkpoint to the `project02_checkpoint` assignment.

* The final due date is on **Wednesday Feb 27 at Noon**. The project should be:
    - submitted to the gradescope programming assignment `project02` and 
    - **saved to pdf** and uploaded to the `project02_free_response` gradescope assignment.

---

Project parts:

1. The first part of project consists of questions much like previous assignments; they will be autograded like all of our assignments. There are many optional "verify these observations for yourself" comments that do not need to be turned in. However, understanding these observations is the entire point of the functions that you're creating and will also help you with part 2 of the project.

2. The second part of the project consists of **self-directed EDA** that builds upon your understanding developed in part 1. This portion of the assignment makes-up **25% of the project grade**. You will be graded not only on the completeness of your EDA, but on the questions and observations themselves. The observations should be compelling, reasoned, and supported with data.

AGAIN: **DO NOT MISS THE FREE RESPONSE AT THE BOTTOM OF THE NOTEBOOK**

In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
%matplotlib inline
import folium
import seaborn as sns

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import project02 as proj

# Introduction: College Scorecard Data

This project looks at the [College Scorecard Data](https://collegescorecard.ed.gov/) that we briefly look at in HW02.

The data you'll be using is found in the `data` directory of the project, but you can find both more data as well as extensive documentation [here](https://collegescorecard.ed.gov/data/).

The data contain extensive information of every post-secondary institution of learning in the United States. 
* Part 1 of this project will explore this college landscape through the basic and most important fields in the dataset.
* Part 2 consists of a **self-directed** EDA project.

While some of the fields in the dataset are easily understood upon reading the data dictionary, there are two very important fields that describe *foundational attributes* of the college landscape in the US. In the dataset, these are:
1. `CONTROL`: The control-type of the institution. Whether the school is a publicly funded institution (like UCSD), a private non-profit institution (like Harvard), or a for-profit institution (like the University of Phoenix). For-profit institutions in particular have often been accused of financially taking advantage of their student-body.
2. `PREDDEG`: The predominant degree awarded at the institution. These are either (1) certificate programs, like trade schools, (2) 2-year associate degrees (awarded from junior colleges), (3) bachelor degree granting institutions, and (4) graduate degree granting institutions (excluded from our data).

These fields are *very* important to understanding the line of questions that the project takes. I'd encourage you to do some outside research if you don't know about these topics.

For example:
* The [US student loan crisis](https://www.nytimes.com/interactive/2018/08/25/opinion/sunday/student-debt-loan-default-college.html)
* [Beauty schools and student debt](https://www.nytimes.com/2018/12/26/business/cosmetology-school-debt-iowa.html)

Below, the dataset is imported for you:

In [6]:
college_path = os.path.join('data', 'MERGED2016_17_PP.csv')
college = pd.read_csv(college_path)

# The Data Dictionary

First, we need to understand what is in the dataset. There is an excel file with a description of what's in the dataset. Open it *in Pandas* and take at its structure. 

The college scorecard is a very *wide* dataset; there are ~7000 schools, with over 100 columns. In the "full data" there are over 1000 columns. Being able to search and parse the data dictionary with Pandas is very necessary for working with the full data!

In [7]:
datadict_path = os.path.join('data', 'CollegeScorecardDataDictionary.xlsx')

datadict = pd.read_excel(datadict_path, sheet_name='data_dictionary')

As observed in homework 2, the college scorecard dataset has a number of 'integer-coded nominal fields'. Two of the most important codes are:
* `PREDDEG` is the "predominant degree type awarded" by the college, and
* `CONTROL` is the "control of the institution" (e.g. public school).

The mapping of integers ("value") to descriptions ("label") is given in the data-dictionary. We would like to replace the integers-codes in the `college` dataset with their string values to make it more readable for our visualization and analysis. 

**Question 1**

While this question is a repeat of a question from HW02, you should try to improve your answer from that HW. In particular you should write your solution *without* loops - this question is good midterm practice!

Create a function `translation_dict` that takes in a dataframe of the form of `datadict` and outputs a dictionary satisfying the following conditions:

1. The keys are the column names of `colleges` that are strings encoded as integers (i.e. columns for which `VALUE` and `LABEL` in `datadict` are non-empty).
2. The values are also dictionaries; each has keys given by `VALUE` and values `LABEL`.

*Note*: This time, **do *not* use loops** in `translation_dict`. The `fillna` and `groupby` methods should help do this!

Once this dictionary is constructed, it can be fed into Pandas DataFrame `.replace` method.

# College Scorecard EDA: Understanding the US college landscape

First we want to understand what the state of post-secondary education looks like in the US. We'd like to know things like:
1. How many schools are there? what types of institutions are they? where are they?
2. How many students attends these schools? What sort of institutions do most students attend?
3. What are these school teaching students? Do most students attend a trade school or a classic university?

In this preliminary EDA section, we will largely concern ourselves with the following *core* columns given in the list `basic_cols` (found in `project02.py` and imported below).

In [9]:
basic_cols = proj.BASIC_COLS

In [10]:
college[basic_cols].head().T

,0,1,2,3,4
UNITID,100654,100663,100690,100706,100724
OPEID,100200,105200,2503400,105500,100500
OPEID6,1002,1052,25034,1055,1005
INSTNM,Alabama A & M University,University of Alabama at Birmingham,Amridge University,University of Alabama in Huntsville,Alabama State University
ZIP,35762,35294-0110,36117-3553,35899,36104-0271
LATITUDE,34.7834,33.5057,32.3626,34.7246,32.3643
LONGITUDE,-86.5685,-86.7993,-86.174,-86.6404,-86.2957
CONTROL,1,1,2,1,1
PREDDEG,3,3,3,3,3
UGDS,4616,12047,293,6346,4704


The dataset has a few identification numbers that are useful to understand:

- `UNITID` is the unique identifier for an organization or school name given by the Integrated Postsecondary Education Data System (IPEDS). It is a unique in this dataset (check this!)
- `OPEID` is the Office of Postsecondary Education Identification code. It is a unique identifier for a school and is unique in this dataset (check this!)
- `OPEID6` is the OPEID code *without the last two digits*. Schools with the same `OPEID6` are run under the same umbrella organization. Look up schools with the same `OPEID6` and compare their names.

**Question 2**

Compute the following statistics on the dataset:
* How many schools are there?
* How many [satellite campuses](https://en.wikipedia.org/wiki/Satellite_campus) are there? (look at the `OPEID` columns; this should include the main campus)
* How many undergraduate students are there in the US?
* What is the average (mean) student population of a university in the US?

Create a function `basic_stats` that takes in `college` and returns a Series of the above statistics index by:
```
['num_schools', 'num_satellite', 'num_students', 'avg_univ_size']
```

Plot a histogram of the number of schools of a given size (in students) consisting of 20 bins and a log-scaled y-axis. Create this plot via a function `plot_school_sizes` that takes in `college` and returns a `matplotlib.axes` object.

**Question to consider (not graded)**: Does it make sense to aggregate the dataset so that the individual datum being considered is a single campus (i.e. a row in `college`) or a campus including it's satellites? What are two lines of investigation that would require considering one versus the other?

**Question 3**

Now we'd like to understand the extent to which the colleges are skewed toward small schools versus large schools. This question will answer "how many of the smallest schools does it take to equal the size of the `k` largest schools?"

Create a function `num_of_small_schools` that takes in `college` and a number `k`, and returns the number of the smallest schools in `college` whose undergraduate student bodies sum to the size of the `k` largest schools.

*Hint:* You may find the `.cumsum()` method useful!

## Control Type and Predominant Degree

**Question 4**

Now we want to understand two of the most influential fields in the dataset: `CONTOL` and `PREDDEG`.

* Create a function `col_pop_stats` that takes in `college` and a categorical column `col`, outputs a dataframe indexed by values of `col` with columns:
    - `size` representing the number of colleges in each category of `col`,
    - `sum` representing the total undergraduate population in each category of `col`,
    - `mean` representing the mean school size in each category of `col`
    - `median` representing the median school size in each category of `col`.
    
* Create a function `col_pop_stats_plot` that takes in a dataframe like the output of `col_pop_stats` and the dataframe `datadict`, returning a `matplotlib.axes` that produces a plot as below.

<img src="imgs/q4.png" width="25%">

Use your plot to understand the `CONTROL` and `PREDDEG` columns. Which colleges tend to be large? Which ones tend to be small? Verify these for yourself!

**Question 5**

* Create a function `control_preddeg_stats` that takes in `college` and a function $f:\text{pd.Series}\to\text{value}$, and outputs a dataframe:
    - indexed by values of `CONTROL`
    - with columns labeled by values of `PREDDEG`
    - the values of which are $f$ applied to `UGDS` for colleges of the given `CONTROL` and `PREDDEG`.

For example, if $f$ is `lambda x:x.sum()`, then the output would be a dataframe giving the number of undergraduates attending each `CONTROL`,`PREDDEG` pair.

* Create a function `control_preddeg_stats_plot` that takes in a dataframe like the output of `control_preddeg_stats`, returning a `matplotlib.axes` that produces a plot as below.

<img src="imgs/q5.png" width="25%">


Use this to understand the `CONTROL` and `PREDDEG` with respect to:
1. the number of colleges for each pair,
2. the number of students attending each pair,
3. the median size of a college of each `CONTROL`/`PREDDEG` combination,
3. the mean size of a college of each `CONTROL`/`PREDDEG` combination.


## State-by-State Analysis

Next we'd like to understand what the college landscape looks like by state. Verify for yourself (non-required) the basic analyses above by state.

**Question 6**

Now, we'd like to understand the *where* the colleges are on a map, where most the public/private/for profit colleges are located, and how large they are. To begin, plot a scatterplot of the colleges via their latitude and longitude in the xy-plane; you should see an outline of the US implicit in the plot.

Specifically, create a function `scatterplot_us` that takes in `college` and outputs a `matplotlib.axes` object that meets the following requirements:
* Each school in the *lower 48 states, plus DC* should be plotted at its latitude/longitude.
* The CONTROL should be encoded by an appropriate set of colors for its kind of data.
* The size of the institution should be coded in dot size.
* The title should be `Undergraduate Institutions`

<img src="imgs/scatterplot.png" width="25%">

*Note:* The CONTROL values have underscores because of a bug in seaborn that tries to coerce a value to float to (incorrectly) determine if the data is quantitative or categorical.

### Percent of state population

**Question 7**

It's natural that some states would have more schools than others, as certain states are larger than others. In this question, use `folium` to plot a choropleth map of the number of college as a percentage of each states population. That is, plot a map where each state is colored by the following ratio: 
```
(number of college students in state) / (number of residents in state)
```
This is *not* the percentage of the state population that's enrolled in college, due to that fact that college students are often not residents of the state. However, it is still a reasonable proxy, as larger states do have more of residents to educate.

Your folium map should look similar to the unemployment example in lecture; your bins should be given by the standard quartiles.

Create a function of zero variables `plot_folium` that returns a dictionary keyed by the `keyword args` for the `choropleth` method, and whose values are the inputs for creating your map. You function should read in all the necessary data from disk (you may assume the data will be in the `data` directory).

Additionally, save your map to a file in the same directory as your `project02.py` called `pct_students_by_state.html` and be sure it's uploaded to Gradescope.


In [25]:
# read in data/state_fips
fips = ...

In [26]:
# read in census populations: data/population-2017.csv
states = ...

In [43]:
# read in geojson file data/gz_2010_us_040_00_5m.json
state_geo = ...

In [27]:
# plot your map and save it to file!

**Question 8**

Next, we would like to address whether certain states create more favorable conditions for one control type over others. We will investigate this by calculating the percentage of the colleges in each state by control-type.

* That is, create a function `control_type_by_state` that takes in `college` and outputs the empirical distribution of `CONTROL` type conditional on state -- i.e. a dataframe indexed by `STABBR`, with columns labeled by the values of `CONTROL`. Include only college is the lower 48 states and Washington DC.

* Now calculate the TVD of the conditional distributions with respect to the unconditional distribution of `CONTROL` across all colleges in the lower 48 states. Do this in a function `tvd_states` that takes in `college` and outputs a series indexed by states with values the TVD for each state, sorted by `tvd` in descending order.

Take note which states are most unlike the 'average' state. *How are they different than the rest?*

### Trade Schools

Many schools are "specialty" schools that only have a few areas of focus for students to study. These may be trade schools (e.g. culinary school) or schools that only focus on business and accounting. In this question, we will focus on understanding the quantity of specialty schools in the US and what their specialties they focus on.

The subjects taken at a school are given in the `PCIPXX` columns where `XX` is a two digit number. Look in the data dictionary at the meaning of these fields.

**Question 9**


For the preliminary steps to doing the next question:
Create a list of `PCIP` columns and slice off only those columns and plot the number of schools that have students taking at least `k` subjects (that is, k subjects columns are non-zero). What does this distribution look like? Is there a natural break?


* Create a function `num_subjects` that takes in `college` and outputs a Series, indexed by `college.index`, which gives the number of subjects the school has at least one student taking.

* Create a function `subject_counts` that takes in `college` and outputs a dataframe that contains the number of students attending schools of each control-type that offers `k` subjects. The output dataframe should:
    - be indexed by the number of subjects offered at the school,
    - have columns given by the three control types.

Plot the output table for your own understanding (which control type has the most specialty school enrollment). Do the same for `PREDDEG` as well (ungraded).

Define a specialty school as being a school that has students only taking focusing on a single subject. Now, we want to create a column that lists the specialty of the given schools.

* Create a function `create_specialty` that takes in `college` and `datadict` and outputs a copy of `college` with an additional column called `SPECIALTY` that gives the specialty of a school. The value of the specialty should be the English description from the data dictionary (`LABEL`), not the code given by the column name (`VARIABLE`). If a school is not a specialty school, the value should be `NaN`.

Take the output and verify the most common type of specialties among specialty schools? Additionally, how many students are enrolled at these specialty schools? (Verify for yourself).

### Completion Rate and Missingness (Extra Credit)

**Please try me!**
This extra credit is not too difficult and has opportunities for partial credit. It's given as extra credit largely because of the assignment length. It's a good opportunity for extra points!

The completions rate for first-time, full-time students within 150% of the normal completion time is given by the field `C150_4_POOLED_SUPP`. This field is missing almost 60% of the time.

We are interested in whether this completion rate is correlated to the amount of student debt students are paying off (given by `GRAD_DEBT_MDN_SUPP`). However, we need to fill in the missing values for the completion rate. Follow the steps below:
1. Drop null values for `GRAD_DEBT_MDN_SUPP` to simplify things (very few are null).
2. Verify that `C150_4_POOLED_SUPP` depends on `CONTROL` and `PREDDEG` via plots of conditional distributions.
3. Do the imputation portion of a multiple imputation on `CONTROL` and `PREDDEG`. Impute from the empirical distributions 50 times for each variable (`CONTROL` and `PREDDEG`). Create a function `impute` that takes in `college` and outputs a dataframe of imputed values for the column `C150_4_POOLED_SUPP` as described above.
4. Plot the distribution of means of the `C150_4_POOLED_SUPP` after imputation (i.e. the 100 means from above).
5. Compute the 100 correlation coefficients from the imputations above and return the 95% confidence interval for those coefficients. Create a function `correlation_bootstrap` which takes in the imputed values from step 3 and returns the 95% confidence interval as described above.

# Self-Directed EDA

## Graded as 25% of the project

In this section, you should pick some columns beyond the basic columns (in `basic_cols`), perform some EDA on those columns, and follow a line of investigation to make some interesting observations. Choose your line of investigation from the suggestions at the bottom of the cell.

You may expand upon the sample questions as much as you'd like. Upon choosing a line of investigation, you are encouraged to do a little background research to understand the context of the problem.

You should follow the guidelines for EDA in lecture and follow them! 
- You should engage in univariate and bivariate analysis.
- Your plots should be understandable from just reading them (e.g. use descriptive titles, substitute integer codes for descriptions). 
- You should write **a summary paragraph** of your findings at the top of this section. Think of this as a thesis for a term paper. This paragraph should refer to evidence and analyses of the data performed below it. It should:
    1. Address a question that is interesting enough to involve connections between multiple attributes,
    2. State your question and summarize the findings of your investigation.
    3. Address the shortcoming of your analysis (perhaps because of lack of information).
    4. Address correlations and possible causation between variables, discussing possible confounders.
- Include your work below the summary. Keep your work and code *clean* and **don't include long table outputs**. 
- **The only output that you display should be output you want to be graded on!**
    
Address one of the follow lines of investigation (expanding on them as you'd like!):
* Begin with addresses one of the 'optional' questions addressed in the project and expand on it.
* Connections between federally subsidized student loans and repayment/default rates. Are there institutions that seem to abuse the student loan program? Address confounders.
* Connections between financial data (cost of tuition) and completion rates. How do loans fit into this picture?
* Connections between SAT scores and completion rates. Where might you want to go to school given you know your SAT score? That is, what is the size, school type, and control of your 'ideal' institution. Address possible confounders.
* Connections between the primary subjects studied (`PCIP`) and future earnings. In particular, can you understand the relationship between `CONTROL` type and future earnings while conditioning on the subject studied?
* Who pays back their loans successfully? Can you connect this to a type of school? Be sure to address the economic background of students who typically attend such schools.

Be sure to address your question on both a *school level*, as well as on a *student level*. Refer to the one that's most appropriate for your analysis (as we did in Part I).

---

**Enter Your Text Summary Here / Display your work below**

---